In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *

In [ ]:
# Read the Root Files
path = "/cave/alice-tpc-qc/data/2023/LHC23zx/cpass0/543442.root"
file = ROOT.TFile.Open(path,"READ")

In [ ]:
clustersData = file.TPC.Clusters.At(0).getObject().getClusters()

In [ ]:
nClusters = clustersData.getNClusters()

In [ ]:
coords = []
for pad in ROOT.o2.tpc.painter.getPadCoordinatesSector():
    coords.append([(pad.xVals[0]+pad.xVals[1])/2,(pad.yVals[0]+pad.yVals[1])/2])

In [ ]:
ROOT.o2.tpc.painter.getPadCoordinatesSector()[0].yVals

In [ ]:
angles = np.linspace(0,360,19)[1:]
angles -= angles[0]/2

In [ ]:
%jsroot on
canvas = ROOT.TCanvas("","",600,600)
hist = ROOT.TH2F("coords","coords",332,-300,300,332,-300,300)
for sector in range(18):
    for pad in ROOT.o2.tpc.painter.getPadCoordinatesSector():
        pad.rotate(angles[sector])
        #coords.append([pad.xVals[0],pad.yVals[0]])
        hist.Fill((pad.xVals[0]+pad.xVals[1])/2,(pad.yVals[0]+pad.yVals[2])/2)
hist.Draw("COLZ")
canvas.Draw()

In [ ]:
%jsroot on
rVals = [sqrt(pad.xVals[0]**2+pad.yVals[0]**2) for pad in ROOT.o2.tpc.painter.getPadCoordinatesSector()]
phiVals = [math.atan(pad.yVals[0]/pad.xVals[0])+math.pi/18 for pad in ROOT.o2.tpc.painter.getPadCoordinatesSector()]

canvas = ROOT.TCanvas("","",800,400)
reference = ROOT.TH2F("reference","reference",180,0,2*math.pi,100,80,255)
hist = ROOT.TH2F("vals","vals",180,0,2*math.pi,100,80,255)
nClusters.getData()[0].getData()
for sector in range(18):
    for pad in range(len(rVals)):
        reference.Fill(phiVals[pad]+(sector/9*math.pi),rVals[pad])
        
reference.Draw("COLZ")
canvas.Draw()

In [ ]:
%jsroot on
canvas = ROOT.TCanvas()
hist = ROOT.TH2F("coords","coords",100,-200,200,100,-200,200)
for i,padVal in enumerate(nClusters.getData()[0].getData()):
    hist.Fill(coords[i][0],coords[i][1],padVal)
hist.Draw("COLZ")
canvas.Draw()

In [ ]:
nClusters.getData()[0].getValue()

In [ ]:
mapper = ROOT.o2.tpc.Mapper.instance()

In [ ]:
mapper.getNumberOfPadsInRowSector(500)

In [ ]:
pads = 0
for row in range(mapper.getNumberOfRowsInIROC()):
    #for sector in range(18):
    pads += mapper.getNumberOfPadsInRowSector(row)
pads